In [59]:
import struct

#changed
def float_to_binary(value):
    # 获取浮点数的IEEE 754标准二进制表示
    binary = ''.join(format(b, '08b') for b in bytearray(struct.pack('>f', value)))
    # 提取符号位、指数位和尾数位
    if (binary[1:] == '0' * 31):
        sign = '0'
        exponent = '100000000'
        mantissa = '0' + binary[9:]
    else:
        sign = str(binary[0])
        exponent = binary[1:9]
        mantissa = "1" + binary[9:]

    return sign, exponent, mantissa

def shift_exponent(binary):
    if binary.startswith("1"):
        shift = '0' + binary[1:].zfill(7)
    else:
        shift = '1' + binary[1:].zfill(7)

    return shift

def shift_back_exponent(binary):
    if binary[0] == "1":
        shift = '0' + binary[1:].zfill(7)
    elif binary[0] == "0":
        shift = '1' + binary[1:].zfill(7)
    return shift

def binary_to_binary_complement16(signal, binary):
    if (signal == '1'):
        #按位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in binary)
        #最后一位加一
        complement = str(bin(int(inverted, 2))[2:].zfill(16))[-16:] # 反码计算
        mantissa = signal + signal + complement
    else:
        mantissa = signal + signal + binary
    return mantissa

def binary_to_float(binary):
    padded_binary = binary.zfill(32)
    byte_string = bytes(int(padded_binary[i:i+8], 2) for i in range(0, 32, 8))
    value = struct.unpack('>f', byte_string)[0]
    return value

def Exponent_MUlTI(a_exponent, b_exponent):
    a_exponent = a_exponent[0] + a_exponent
    b_exponent = b_exponent[0] + b_exponent
    ab_exponent = str(bin(int(a_exponent, 2) + int(b_exponent, 2))[2:]).zfill(9)[-9:]
    return ab_exponent

#changed
def exponent_judge(exponent, mantissa): #上溢
    if exponent.startswith('01') or exponent[2:] == '1111111':
        exponent = '011111110'
        if mantissa.startswith('0'):
            mantissa = '0' + '1' * 25
        else:
            mantissa = '1' + '00000000000' * 24 + '1'
    elif exponent.startswith('10'): #下溢
        exponent = '100000000'
        mantissa = '0' * 26
    else:
        exponent = exponent
        mantissa = mantissa
    return exponent, mantissa

def INPUT (float):
    sign, exponent, mantissa = float_to_binary(float)
    exponent = shift_exponent(exponent)
    return  sign, exponent, mantissa

def MULTI(a_sign, a_mantissa, b_sign, b_mantissa):
    ab_sign = str(int(a_sign, 2) ^ int(b_sign, 2))
    ab_mantissa = str(bin(int(a_mantissa, 2) * int(b_mantissa, 2))[2:].zfill(16))   # 乘法
    ab_mantissa = binary_to_binary_complement16(ab_sign, ab_mantissa)               # 转补码 因为要将结果拼接所以不能舍入处理
    return ab_mantissa

def Splicing_PE (Exp_in, low_mantissa, high_mantissa):
    if(Exp_in == '1'):
        low_mantissa =  low_mantissa[0] * 7  + low_mantissa
    else:
        low_mantissa =  low_mantissa[0] * 8  + low_mantissa      
        
    S_mantissa = str(bin(int(low_mantissa[0:18], 2) + int(high_mantissa, 2))[2:].zfill(18))[-18:]
    S_mantissa = S_mantissa + low_mantissa[18:26]
    Exp = '0'

    if S_mantissa.startswith('01'):
        S_mantissa = '0' + S_mantissa[:-1]
        Exp = '1'
    elif S_mantissa.startswith('10'):
        S_mantissa = '1' + S_mantissa[:-1]
        Exp = '1'

    return Exp, S_mantissa


In [60]:
#与两个尾数的移位有关系，所以可能要修改 
def exponent_judge_32(exponent, mantissa): #上溢
    if exponent.startswith('01') or exponent[2:] == '1111111':
        exponent = '011111110'
        if mantissa.startswith('0'):
            mantissa = '0' + '1' * 25
        else:
            mantissa = '1' + '00000000000' * 24 + '1'
    elif exponent.startswith('10'): #下溢
        exponent = '100000000'
        mantissa = '0' * 26
    else:
        exponent = exponent
        mantissa = mantissa
    return exponent, mantissa

def shift_exponent_32(binary):
    if binary.startswith("1"):
        shift = '0' + binary[1:].zfill(7)
    else:
        shift = '1' + binary[1:].zfill(7)

    return shift

def binary_to_float_32(binary):
    padded_binary = binary.zfill(32)
    byte_string = bytes(int(padded_binary[i:i+8], 2) for i in range(0, 32, 8))
    value = struct.unpack('>f', byte_string)[0]
    return value

def binary_to_binary_complement_32(signal, binary):
    if (signal == '1'):
        #按位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in binary)
        complement = str(bin(int(inverted, 2) + 1)[2:].zfill(24))[-24:]  
        mantissa = signal + signal + complement
    else:
        mantissa = signal + signal + binary
    return mantissa

def Exponent_Shift_32(a_exponent, b_exponent):
    if a_exponent.startswith('10'):
        if b_exponent.startswith('10'):
            max_exponent = '100000000'
            max_signal = '00'
            shift_exponent = '0'
            overflow = '11'
        else:
            max_exponent = b_exponent
            shift_exponent = '0'
            max_signal = '10'
            overflow = '01'
    elif b_exponent.startswith('10'):
        max_exponent = a_exponent
        max_signal = '01'
        shift_exponent = '0'
        overflow = '10'
    else:
        overflow = '00'
        #因为python笨比的计算机制，所以正数和负数参与的计算的结果不一样，但是实际FPGA中是一样的。
        if(a_exponent.startswith('1') and b_exponent.startswith('0')):
            shift_exponent = str(bin(int(b_exponent, 2) - int(a_exponent, 2)))
        else:
            shift_exponent = str(bin(int(a_exponent, 2) - int(b_exponent, 2)))

        if(a_exponent[0] == b_exponent[0]): 
            if(a_exponent[0] == '0'): 
                #这里是被迫采用这种判断方法的，因为python无法计算二进制正数的补码运算
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(9, '0'))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent  
                        max_signal = '10'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(9))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent
                        max_signal = '01'
                
                shift_exponent = int(shift_exponent, 2)

            else:
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(9, '1'))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent  
                        max_signal = '01'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(9))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent
                        max_signal = '10'

                if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:9])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(8))[-8:]  # 加1得到补码
                    shift_exponent = '0' + complement
                shift_exponent = int(shift_exponent, 2)
        else:
            if(shift_exponent.startswith('-')):
                shift_exponent = str(shift_exponent[3:].rjust(9, '1'))
            else:
                shift_exponent = str(shift_exponent[2:].zfill(9))

            if(a_exponent[0] == '0'):
                max_exponent = a_exponent
                max_signal = '01'
            else:
                max_exponent = b_exponent
                max_signal = '10'

            if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:9])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(8))[-8:]  # 加1得到补码
                    shift_exponent = '0' + complement
            shift_exponent = int(shift_exponent, 2)
    return shift_exponent, max_exponent, max_signal, overflow

def ADDER_32(Proga, Progb, a_mantissa, b_mantissa, shift_exponent, max_signal, overflow):
    if(overflow == '11'):
        ab_mantissa = '0' * 26 #修改
        Prog = '0'
        Inst = '11' #跳过
    elif (overflow == '01'):
        ab_mantissa = b_mantissa
        Prog = Progb
        Inst = '11' #跳过
    elif (overflow == '10'):
        ab_mantissa = a_mantissa
        Prog = Proga
        Inst = '11' #跳过
    else:
        if(max_signal == '00'):#A B相同的半区
            if int(Proga, 2) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(Proga, 2) < int(Progb, 2):
                Prog = Progb
                shift = str(int(Progb, 2) - int(Proga, 2))
                Inst = '01' #A对位
            else:
                Prog = Proga
                shift = str(int(Proga, 2) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '01'): #A大的半区
            shift = str(int(Proga, 2) + int(shift_exponent))
            if int(shift) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Progb, 2):
                Prog = str(bin(int(Proga ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '01' #A对位 
            else:
                Prog = Proga
                shift = str(int(shift) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '10'): #B大的半区
            shift = str(int(Progb, 2) + int(shift_exponent))
            if int(shift) == int(Proga, 2):
                Prog = Progb
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Proga, 2):
                Prog = str(bin(int(Progb ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '10' #B对位
            else:
                Prog = Progb
                shift = str(int(shift) - int(Proga, 2))
                Inst = '01' #A对位
        else:
            shift = '0'
            Inst ='11'
    
    if Inst == '00': # 同为加
        a = a_mantissa
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
    elif Inst == '01': # A对位 B不动
        if(int(shift) < 26 ):
            a = a_mantissa[0] * int(shift) + a_mantissa[0:26-int(shift)]
        else:
            a = a_mantissa[0] * 26
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
    elif Inst == '10': # B对位 A不动
        if(int(shift) < 26 ):
            b = b_mantissa[0] * int(shift) + b_mantissa[0:26-int(shift)]
        else:
            b = b_mantissa[0] * 26
        a = a_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
    else:
        Prog_out = Prog
        ab_mantissa = ab_mantissa

    if ab_mantissa.startswith('01'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:25]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])
    elif ab_mantissa.startswith('10'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:25]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])

    return Prog_out, ab_mantissa

def OUTPUT_32(sign_result, exponent_result, mantissa_result):
    #需要增加一个计算00停止的条件
    while mantissa_result[0:3] == "000" or mantissa_result[0:3] == "111":
        mantissa_result = str(mantissa_result[1:26]) + '0'  # 数据左移 指数减少
        exponent_result = str(bin((int(exponent_result, 2) - 1) & 0x1FF))[2:].zfill(9) # 指数位减 1
        if(mantissa_result == '1' * 26 or mantissa_result == '0' * 26):
            exponent_result = '100000000'
            break

    while mantissa_result[0:2] == "01" or mantissa_result[0:2] == "10":
        mantissa_result = mantissa_result[0] + mantissa_result[0:25] # 数据右移 指数增加
        exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1
        
    exponent_result, mantissa_result = exponent_judge_32(exponent_result, mantissa_result)

    #指数转换
    if exponent_result[0] == '1':
        exponent_result = '0' + exponent_result[2:9]

    else:
        exponent_result = '1' + exponent_result[2:9]

    #补码转原码
    if sign_result == '1':
        # 对补码的符号位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in mantissa_result)  
        # 加1得到原码
        mantissa_result = bin(int(inverted, 2) + 1)[2:].zfill(26)
    else:
        mantissa_result = mantissa_result

    result = str(sign_result + exponent_result + mantissa_result[3: 26])
    return result

def IEEE754_ADD_32(fA, fB):
    # 改
    A1_exponent = shift_exponent_32(fA[1:9])
    if (fA[1:] == '0' * 31):
        A1_sign = '0'
        A1_mantissa = '0' + fA[9:]
    else:
        A1_sign = str(fA[0])
        A1_mantissa = "1" + fA[9:]
    A1_mantissa = binary_to_binary_complement_32(A1_sign, A1_mantissa)
    # 改
    B1_exponent = shift_exponent_32(fB[1:9])
    if (fB[1:] == '0' * 31):
        B1_sign = '0'
        B1_mantissa = '0' + fB[9:]
    else:
        B1_sign = str(fB[0])
        B1_mantissa = "1" + fB[9:]
    B1_mantissa = binary_to_binary_complement_32(B1_sign, B1_mantissa)

    Shift, Exponent, max_signal, overflow = Exponent_Shift_32(A1_exponent, B1_exponent)

    Prog, Mantissa = ADDER_32('0', '0', A1_mantissa, B1_mantissa, Shift, max_signal, overflow)

    Signal = Mantissa[0]
    Exponent = str(bin(int(Exponent, 2) + int(Prog, 2))[2:].zfill(9))

    result = OUTPUT_32(Signal, Exponent, Mantissa)
    return result

def IEEE754_MUL_32(fA, fB):
    # 改
    A1_exponent = shift_exponent_32(fA[1:9])
    if (fA[1:] == '0' * 31):
        A1_sign = '0'
        A1_mantissa = '0' + fA[9:]
    else:
        A1_sign = str(fA[0])
        A1_mantissa = "1" + fA[9:]

    # 改
    B1_exponent = shift_exponent_32(fB[1:9])
    if (fB[1:] == '0' * 31):
        B1_sign = '0'
        B1_mantissa = '0' + fB[9:]
    else:
        B1_sign = str(fB[0])
        B1_mantissa = "1" + fB[9:]

    A1_exponent = A1_exponent[0] + A1_exponent
    B1_exponent = B1_exponent[0] + B1_exponent       

    Exponent = str(bin(int(A1_exponent, 2) + int(B1_exponent, 2))[2:]).zfill(9)[-9:]
    Exponent = str(bin((int(Exponent, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1
    Exponent = str(bin((int(Exponent, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1  
    
    # 改 位数47 24 + 24 = 48 48 - 1 = 47
    ab_sign = str(int(A1_sign, 2) ^ int(B1_sign, 2))
    ab_mantissa = str(bin(int(A1_mantissa, 2) * int(B1_mantissa, 2))[2:].zfill(48))   # 乘法

    Mantissa = binary_to_binary_complement_32(ab_sign, ab_mantissa[0:24]) 

    Signal = Mantissa[0]
    result = OUTPUT_32(Signal, Exponent, Mantissa)
    return result

In [61]:
#输入
floatA1 = -3.4028234663852886e+38
floatB1 = 2
floatA2 = 191.0023439976028
floatB2 = -36.1981478612899
floatA3 = -78.7887748948242
floatB3 = -61.85795420155745

A1_sign, A1_exponent, A1_mantissa = INPUT(floatA1)
B1_sign, B1_exponent, B1_mantissa = INPUT(floatB1)

A1H = A1_mantissa[0:8]
A1M = A1_mantissa[8:16]
A1L = A1_mantissa[16:24]

B1H = B1_mantissa[0:8]
B1M = B1_mantissa[8:16]
B1L = B1_mantissa[16:24]

A2_sign, A2_exponent, A2_mantissa = INPUT(floatA2)
B2_sign, B2_exponent, B2_mantissa = INPUT(floatB2)

A2H = A2_mantissa[0:8]
A2M = A2_mantissa[8:16]
A2L = A2_mantissa[16:24]

B2H = B2_mantissa[0:8]
B2M = B2_mantissa[8:16]
B2L = B2_mantissa[16:24]

A3_sign, A3_exponent, A3_mantissa = INPUT(floatA3)
B3_sign, B3_exponent, B3_mantissa = INPUT(floatB3)

A3H = A3_mantissa[0:8]
A3M = A3_mantissa[8:16]
A3L = A3_mantissa[16:24]
B3H = B3_mantissa[0:8]
B3M = B3_mantissa[8:16]
B3L = B3_mantissa[16:24]

#相当于每一个PE内部的数据进行处理，相当于硬件中把每一个PE内部的数据的都求出来
#第一组
A1LBL_mantissa = MULTI(A1_sign, A1L, B1_sign, B1L)
A1LBM_mantissa = MULTI(A1_sign, A1L, B1_sign, B1M)
print('A1M = {}'.format(A1M))
print('B1L = {}'.format(B1L))
print('B1L = {}'.format(B1M))
print('B1L = {}'.format(B1H))
A1LBH_mantissa = MULTI(A1_sign, A1L, B1_sign, B1H)
A1MBL_mantissa = MULTI(A1_sign, A1M, B1_sign, B1L)
A1MBM_mantissa = MULTI(A1_sign, A1M, B1_sign, B1M)
A1MBH_mantissa = MULTI(A1_sign, A1M, B1_sign, B1H)
A1HBL_mantissa = MULTI(A1_sign, A1H, B1_sign, B1L)
A1HBM_mantissa = MULTI(A1_sign, A1H, B1_sign, B1M)
A1HBH_mantissa = MULTI(A1_sign, A1H, B1_sign, B1H)
#第二组
A2LBL_mantissa = MULTI(A2_sign, A2L, B2_sign, B2L)
A2LBM_mantissa = MULTI(A2_sign, A2L, B2_sign, B2M)
A2LBH_mantissa = MULTI(A2_sign, A2L, B2_sign, B2H)
A2MBL_mantissa = MULTI(A2_sign, A2M, B2_sign, B2L)
A2MBM_mantissa = MULTI(A2_sign, A2M, B2_sign, B2M)
A2MBH_mantissa = MULTI(A2_sign, A2M, B2_sign, B2H)
A2HBL_mantissa = MULTI(A2_sign, A2H, B2_sign, B2L)
A2HBM_mantissa = MULTI(A2_sign, A2H, B2_sign, B2M)
A2HBH_mantissa = MULTI(A2_sign, A2H, B2_sign, B2H)
#第三组
A3LBL_mantissa = MULTI(A3_sign, A3L, B3_sign, B3L)
A3LBM_mantissa = MULTI(A3_sign, A3L, B3_sign, B3M)
A3LBH_mantissa = MULTI(A3_sign, A3L, B3_sign, B3H)
A3MBL_mantissa = MULTI(A3_sign, A3M, B3_sign, B3L)
A3MBM_mantissa = MULTI(A3_sign, A3M, B3_sign, B3M)
A3MBH_mantissa = MULTI(A3_sign, A3M, B3_sign, B3H)
A3HBL_mantissa = MULTI(A3_sign, A3H, B3_sign, B3L)
A3HBM_mantissa = MULTI(A3_sign, A3H, B3_sign, B3M)
A3HBH_mantissa = MULTI(A3_sign, A3H, B3_sign, B3H)

print('A1LBL_mantissa = {}'.format(A1LBL_mantissa))
print('A1LBM_mantissa = {}'.format(A1LBM_mantissa))
print('A1LBH_mantissa = {}'.format(A1LBH_mantissa))
print('A1MBL_mantissa = {}'.format(A1MBL_mantissa))
print('A1MBM_mantissa = {}'.format(A1MBM_mantissa))
print('A1MBH_mantissa = {}'.format(A1MBH_mantissa))
print('A1HBL_mantissa = {}'.format(A1HBL_mantissa))
print('A1HBM_mantissa = {}'.format(A1HBM_mantissa))
print('A1HBH_mantissa = {}'.format(A1HBH_mantissa))
print('')
print('A2LBL_mantissa = {}'.format(A2LBL_mantissa))
print('A2LBM_mantissa = {}'.format(A2LBM_mantissa))
print('A2LBH_mantissa = {}'.format(A2LBH_mantissa))
print('A2MBL_mantissa = {}'.format(A2MBL_mantissa))
print('A2MBM_mantissa = {}'.format(A2MBM_mantissa))
print('A2MBH_mantissa = {}'.format(A2MBH_mantissa))
print('A2HBL_mantissa = {}'.format(A2HBL_mantissa))
print('A2HBM_mantissa = {}'.format(A2HBM_mantissa))
print('A2HBH_mantissa = {}'.format(A2HBH_mantissa))
print('')
print('A3LBL_mantissa = {}'.format(A3LBL_mantissa))
print('A3LBM_mantissa = {}'.format(A3LBM_mantissa))
print('A3LBH_mantissa = {}'.format(A3LBH_mantissa))
print('A3MBL_mantissa = {}'.format(A3MBL_mantissa))
print('A3MBM_mantissa = {}'.format(A3MBM_mantissa))
print('A3MBH_mantissa = {}'.format(A3MBH_mantissa))
print('A3HBL_mantissa = {}'.format(A3HBL_mantissa))
print('A3HBM_mantissa = {}'.format(A3HBM_mantissa))
print('A3HBH_mantissa = {}'.format(A3HBH_mantissa))

#PE 纵列拼接
#相当于PE内部的计算结果逐步累加，最终传播的到下层的ADDER的这个过程。
#第一组
Exp1L, A1LBLM_mantissa = Splicing_PE('0'  , A1LBL_mantissa,  A1LBM_mantissa)#L
Exp1L, A1LBMH_mantissa = Splicing_PE(Exp1L, A1LBLM_mantissa, A1LBH_mantissa)
Exp1M, A1MBLM_mantissa = Splicing_PE('0'  , A1MBL_mantissa,  A1MBM_mantissa)#M
Exp1M, A1MBMH_mantissa = Splicing_PE(Exp1M, A1MBLM_mantissa, A1MBH_mantissa)
Exp1H, A1HBLM_mantissa = Splicing_PE('0'  , A1HBL_mantissa,  A1HBM_mantissa)#H
Exp1H, A1HBMH_mantissa = Splicing_PE(Exp1H, A1HBLM_mantissa, A1HBH_mantissa)
#第二组
print('')
Exp2L, A2LBLM_mantissa = Splicing_PE('0'  , A2LBL_mantissa,  A2LBM_mantissa)#L
Exp2L, A2LBMH_mantissa = Splicing_PE(Exp2L, A2LBLM_mantissa, A2LBH_mantissa)
Exp2M, A2MBLM_mantissa = Splicing_PE('0'  , A2MBL_mantissa,  A2MBM_mantissa)#M
Exp2M, A2MBMH_mantissa = Splicing_PE(Exp2M, A2MBLM_mantissa, A2MBH_mantissa)

Exp2H, A2HBLM_mantissa = Splicing_PE('0'  , A2HBL_mantissa,  A2HBM_mantissa)#H
Exp2H, A2HBMH_mantissa = Splicing_PE(Exp2H, A2HBLM_mantissa, A2HBH_mantissa)
#第三组
Exp3L, A3LBLM_mantissa = Splicing_PE('0'  , A3LBL_mantissa,  A3LBM_mantissa)#L
Exp3L, A3LBMH_mantissa = Splicing_PE(Exp3L, A3LBLM_mantissa, A3LBH_mantissa)
Exp3M, A3MBLM_mantissa = Splicing_PE('0'  , A3MBL_mantissa,  A3MBM_mantissa)#M
Exp3M, A3MBMH_mantissa = Splicing_PE(Exp3M, A3MBLM_mantissa, A3MBH_mantissa)
Exp3H, A3HBLM_mantissa = Splicing_PE('0'  , A3HBL_mantissa,  A3HBM_mantissa)#H
Exp3H, A3HBMH_mantissa = Splicing_PE(Exp3H, A3HBLM_mantissa, A3HBH_mantissa)

print('Exp1L = {}'.format(Exp1L))
print('Exp2L = {}'.format(Exp2L))
print('Exp3L = {}'.format(Exp3L))
print('A1LBMH_mantissa = {}'.format(A1LBMH_mantissa))
print('A2LBMH_mantissa = {}'.format(A2LBMH_mantissa))
print('A3LBMH_mantissa = {}'.format(A3LBMH_mantissa))

print('Exp1M = {}'.format(Exp1M))
print('Exp2M = {}'.format(Exp2M))
print('Exp3M = {}'.format(Exp3M))
print('A1MBMH_mantissa = {}'.format(A1MBMH_mantissa))
print('A2MBMH_mantissa = {}'.format(A2MBMH_mantissa))
print('A3MBMH_mantissa = {}'.format(A3MBMH_mantissa))

print('Exp1H = {}'.format(Exp1H))
print('Exp2H = {}'.format(Exp2H))
print('Exp3H = {}'.format(Exp3H))
print('A1HBMH_mantissa = {}'.format(A1HBMH_mantissa))
print('A2HBMH_mantissa = {}'.format(A2HBMH_mantissa))
print('A3HBMH_mantissa = {}'.format(A3HBMH_mantissa))


A1M = 10001100
B1L = 11010110
B1L = 10111000
B1L = 11000010
A1LBL_mantissa = 000100010101100010
A1LBM_mantissa = 000011101110101000
A1LBH_mantissa = 000011111011100110
A1MBL_mantissa = 000111010100001000
A1MBM_mantissa = 000110010010100000
A1MBH_mantissa = 000110101000011000
A1HBL_mantissa = 001001010011001100
A1HBM_mantissa = 000111111111110000
A1HBH_mantissa = 001000011011100100

A2LBL_mantissa = 000111101000110100
A2LBM_mantissa = 000001110101100100
A2LBH_mantissa = 000111100100101100
A2MBL_mantissa = 001001111001001111
A2MBM_mantissa = 000010011000010011
A2MBH_mantissa = 001001110011111001
A2HBL_mantissa = 001100101010111111
A2HBM_mantissa = 000011000011000011
A2HBH_mantissa = 001100100100001001

A3LBL_mantissa = 111111111000000001
A3LBM_mantissa = 111111100101111101
A3LBH_mantissa = 111110101110101101
A3MBL_mantissa = 111111100001111111
A3MBM_mantissa = 111110011101111111
A3MBH_mantissa = 111011001101111111
A3HBL_mantissa = 111111010011101100
A3HBM_mantissa = 111101101111010010
A3

In [53]:
def Exponent_Shift(a_exponent, b_exponent):
    #changed
    if a_exponent.startswith('10'):
        if b_exponent.startswith('10'):
            max_exponent = '100000000'
            max_signal = '00'
            shift_exponent = '0'
            overflow = '11'
        else:
            max_exponent = b_exponent
            shift_exponent = '0'
            max_signal = '01'
            overflow = '01'
    elif b_exponent.startswith('10'):
        max_exponent = a_exponent
        max_signal = '10'
        shift_exponent = '0'
        overflow = '10'
    else:
        overflow = '00'
        #因为python笨比的计算机制，所以正数和负数参与的计算的结果不一样，但是实际FPGA中是一样的。
        if(a_exponent.startswith('1') and b_exponent.startswith('0')):
            shift_exponent = str(bin(int(b_exponent, 2) - int(a_exponent, 2)))
        else:
            shift_exponent = str(bin(int(a_exponent, 2) - int(b_exponent, 2)))

        if(a_exponent[0] == b_exponent[0]): 
            if(a_exponent[0] == '0'): 
                #这里是被迫采用这种判断方法的，因为python无法计算二进制正数的补码运算
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(9, '0'))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent  
                        max_signal = '10'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(9))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent
                        max_signal = '01'
                
                shift_exponent = int(shift_exponent, 2)

            else:
                if shift_exponent.startswith('-'):
                    shift_exponent = str(shift_exponent[3:].rjust(9, '1'))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = a_exponent  
                        max_signal = '01'          
                else:
                    shift_exponent = str(shift_exponent[2:].zfill(9))
                    if shift_exponent[1:9] == '00000000':
                        shift_exponent = '000000000'
                        max_exponent = a_exponent
                        max_signal = '00'
                    else:
                        max_exponent = b_exponent
                        max_signal = '10'

                if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:9])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(8))[-8:]  # 加1得到补码
                    shift_exponent = '0' + complement
                shift_exponent = int(shift_exponent, 2)
        else:
            if(shift_exponent.startswith('-')):
                shift_exponent = str(shift_exponent[3:].rjust(9, '1'))
            else:
                shift_exponent = str(shift_exponent[2:].zfill(9))

            if(a_exponent[0] == '0'):
                max_exponent = a_exponent
                max_signal = '01'
            else:
                max_exponent = b_exponent
                max_signal = '10'

            if shift_exponent.startswith('1'):
                    inverted = "".join("1" if bit == "0" else "0" for bit in shift_exponent[1:9])
                    complement = str(bin(int(inverted, 2) + 1)[2:].zfill(8))[-8:]  # 加1得到补码
                    shift_exponent = '0' + complement
            shift_exponent = int(shift_exponent, 2)
    return shift_exponent, max_exponent, max_signal, overflow

In [35]:
print('A1_exponent= {}'.format(A1_exponent))
print('B1_exponent= {}'.format(B1_exponent))
print('A2_exponent= {}'.format(A2_exponent))
print('B2_exponent= {}'.format(B2_exponent))
print('A3_exponent= {}'.format(A3_exponent))
print('B3_exponent= {}'.format(B3_exponent))
#Exponent阵列，每一个PE先计算出每一个PE对应的
#A1
A1B_exponent = Exponent_MUlTI(A1_exponent, B1_exponent)
#A2
A2B_exponent = Exponent_MUlTI(A2_exponent, B2_exponent)
#A3
A3B_exponent = Exponent_MUlTI(A3_exponent, B3_exponent)
print('A1B_exponent = {}'.format(A1B_exponent))
print('A2B_exponent = {}'.format(A2B_exponent))
print('A3B_exponent = {}'.format(A3B_exponent))

Shift12, Exponent_12B, max_signal1, overflow1 = Exponent_Shift(A1B_exponent, A2B_exponent)
Shift23, Exponent_23B, max_signal2, overflow2 = Exponent_Shift(Exponent_12B, A3B_exponent)
print('Shift12      = {}'.format(Shift12))
print('Shift23      = {}'.format(Shift23))
print('Exponent_12B = {}'.format(Exponent_12B))
print('Exponent_23B = {}'.format(Exponent_23B))
print('max_signal1  = {}'.format(max_signal1))
print('max_signal2  = {}'.format(max_signal2))
print('overflow1    = {}'.format(overflow1))
print('overflow2    = {}'.format(overflow2))

A1_exponent= 00001101
B1_exponent= 00001110
A2_exponent= 00001110
B2_exponent= 00001101
A3_exponent= 00001010
B3_exponent= 00001010
A1B_exponent = 000011011
A2B_exponent = 000011011
A3B_exponent = 000010100
Shift12      = 0
Shift23      = 7
Exponent_12B = 000011011
Exponent_23B = 000011011
max_signal1  = 00
max_signal2  = 01
overflow1    = 00
overflow2    = 00


In [36]:
def ADDER(Proga, Progb, a_mantissa, b_mantissa, shift_exponent, max_signal, overflow):
    if(overflow == '11'):
        ab_mantissa = '0' * 26
        Prog = '0'
        Inst = '11' #跳过
    elif (overflow == '01'):
        ab_mantissa = b_mantissa
        Prog = Progb
        Inst = '11' #跳过
    elif (overflow == '10'):
        ab_mantissa = a_mantissa
        Prog = Proga
        Inst = '11' #跳过
    else:
        if(max_signal == '00'):#A B相同的半区
            if int(Proga, 2) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(Proga, 2) < int(Progb, 2):
                Prog = Progb
                shift = str(int(Progb, 2) - int(Proga, 2))
                Inst = '01' #A对位
            else:
                Prog = Proga
                shift = str(int(Proga, 2) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '01'): #A大的半区
            shift = str(int(Proga, 2) + int(shift_exponent))
            if int(shift) == int(Progb, 2):
                Prog = Proga
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Progb, 2):
                Prog = str(bin(int(Proga ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '01' #A对位 
            else:
                Prog = Proga
                shift = str(int(shift) - int(Progb, 2))
                Inst = '10' #B对位
        elif(max_signal == '10'): #B大的半区
            shift = str(int(Progb, 2) + int(shift_exponent))
            if int(shift) == int(Proga, 2):
                Prog = Progb
                shift = str(int(0))
                Inst = '00' #同加
            elif int(shift) < int(Proga, 2):
                Prog = str(bin(int(Progb ,2) + 1)[2:])
                shift = str(int(1))#S
                Inst = '10' #B对位
            else:
                Prog = Progb
                shift = str(int(shift) - int(Proga, 2))
                Inst = '01' #A对位
        else:
            shift = '0'
            Inst ='11'
    
    if Inst == '00': # 同为加
        a = a_mantissa
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
        print('a = {}'.format(a))
        print('b = {}'.format(b))
        # print('ab_mantissa = {}'.format(ab_mantissa))
    elif Inst == '01': # A对位 B不动
        if(int(shift) < 27 ):
            a = a_mantissa[0] * int(shift) + a_mantissa[0:26-int(shift)]
        else:
            a = a_mantissa[0] * 26
        b = b_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
        print('a = {}'.format(a))
        print('b = {}'.format(b))
        # print('ab_mantissa = {}'.format(ab_mantissa))
    elif Inst == '10': # B对位 A不动
        if(int(shift) < 27 ):
            b = b_mantissa[0] * int(shift) + b_mantissa[0:26-int(shift)]
        else:
            b = b_mantissa[0] * 26
        a = a_mantissa
        Prog_out = Prog
        ab_mantissa = str(bin(int(a, 2) + int(b, 2))[2:].zfill(26))[-26:]
        print('a = {}'.format(a))
        print('b = {}'.format(b))
        # print('ab_mantissa = {}'.format(ab_mantissa))
    else:
        Prog_out = Prog
        ab_mantissa = ab_mantissa

    if ab_mantissa.startswith('01'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:25]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])
    elif ab_mantissa.startswith('10'):
        ab_mantissa = ab_mantissa[0] + ab_mantissa[0:25]
        Prog_out = str(bin(int(Prog_out) + 1)[2:])

    return Prog_out, ab_mantissa


In [49]:
#Layer12
Prog12L, Mantissa12L = ADDER(Exp1L, Exp2L, A1LBMH_mantissa, A2LBMH_mantissa, Shift12, max_signal1, overflow1)
Prog12M, Mantissa12M = ADDER(Exp1M, Exp2M, A1MBMH_mantissa, A2MBMH_mantissa, Shift12, max_signal1, overflow1)
Prog12H, Mantissa12H = ADDER(Exp1H, Exp2H, A1HBMH_mantissa, A2HBMH_mantissa, Shift12, max_signal1, overflow1)
print('A1MBMH_mantissa = {}'.format(A1MBMH_mantissa))
print('A2MBMH_mantissa = {}'.format(A2MBMH_mantissa))
print('Prog12L = {}'.format(Prog12L))
print('Mantissa12L = {}'.format(Mantissa12L))
print('Prog3L = {}'.format(Exp3L))
print('A3LBMH_mantissa = {}'.format(A3LBMH_mantissa))

print('Prog12M = {}'.format(Prog12M))
print('Mantissa12M = {}'.format(Mantissa12M))
print('Prog3M = {}'.format(Exp3M))
print('A3MBMH_mantissa = {}'.format(A3MBMH_mantissa))

print('Prog12H = {}'.format(Prog12H))
print('Mantissa12H = {}'.format(Mantissa12H))
print('Prog3H = {}'.format(Exp3H))
print('A3HBMH_mantissa = {}'.format(A3HBMH_mantissa))

a = 00001111110010000111101101
b = 00011110010100100111011110
a = 00011010100111110100010101
b = 00100111010001111110110001
a = 00100001110110010010000100
b = 00110010010011101010001101
A1MBMH_mantissa = 00011010100111110100010101
A2MBMH_mantissa = 00100111010001111110110001
Prog12L = 0
Mantissa12L = 00101110000110101111001011
Prog3L = 0
A3LBMH_mantissa = 11111010111010011001111011
Prog12M = 1
Mantissa12M = 00100000111100111001100011
Prog3M = 0
A3MBMH_mantissa = 11101100110110011001110111
Prog12H = 1
Mantissa12H = 00101010000100111110001000
Prog3H = 0
A3HBMH_mantissa = 11100011101110010111000110


In [38]:
#Layer23
Prog23L, Mantissa23L = ADDER(Prog12L, Exp3L, Mantissa12L, A3LBMH_mantissa, Shift23, max_signal2, overflow2)
Prog23M, Mantissa23M = ADDER(Prog12M, Exp3M, Mantissa12M, A3MBMH_mantissa, Shift23, max_signal2, overflow2)
Prog23H, Mantissa23H = ADDER(Prog12H, Exp3H, Mantissa12H, A3HBMH_mantissa, Shift23, max_signal2, overflow2)
print('Prog23L = {}'.format(Prog23L))
print('Mantissa23L = {}'.format(Mantissa23L))
print('Prog23M = {}'.format(Prog23M))
print('Mantissa23M = {}'.format(Mantissa23M))
print('Prog23H = {}'.format(Prog23H))
print('Mantissa23H = {}'.format(Mantissa23H))

a = 00101110000110101111001011
b = 11111111111101011101001100
a = 00100000111100111001100011
b = 11111111111011001101100110
a = 00101010000100111110001000
b = 11111111111000111011100101
Prog23L = 0
Mantissa23L = 00101110000100001100010111
Prog23M = 1
Mantissa23M = 00100000111000000111001001
Prog23H = 1
Mantissa23H = 00101001111101111001101101


In [39]:
#横列PE结果的拼接处理, ADDER
ProgML, MantissaML = ADDER(Prog23L, Prog23M, Mantissa23L, Mantissa23M, '8', '10', '00')
ProgHM, MantissaHM = ADDER(ProgML,  Prog23H, MantissaML,  Mantissa23H, '8', '10', '00')
print('ProgML = {}'.format(ProgML))
print('MantissaML = {}'.format(MantissaML))
print('ProgHM = {}'.format(ProgHM))
print('MantissaHM = {}'.format(MantissaHM))


a = 00000000000101110000100001
b = 00100000111000000111001001
a = 00000000001000001111011101
b = 00101001111101111001101101
ProgML = 1
MantissaML = 00100000111101110111101010
ProgHM = 1
MantissaHM = 00101010000110001001001010


In [40]:
#指数位拼接
Signal = MantissaHM[0]
Exponent = str(bin(int(Exponent_23B, 2) + int(ProgHM, 2))[2:].zfill(9))
Mantissa = MantissaHM

print('Signal = {}'.format(Signal))
print('Exponent = {}'.format(Exponent))
print('Mantissa = {}'.format(Mantissa))


Signal = 0
Exponent = 000011100
Mantissa = 00101010000110001001001010


In [41]:
#754 Result
fA1 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>f', floatA1))) #32
fB1 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>f', floatB1))) #32
fA2 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>f', floatA2))) #32
fB2 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>f', floatB2))) #32
fA3 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>f', floatA3))) #32
fB3 = ''.join(format(b, '08b') for b in bytearray(struct.pack('>f', floatB3))) #32

A1B1 = IEEE754_MUL_32(fA1, fB1)
A2B2 = IEEE754_MUL_32(fA2, fB2)
A3B3 = IEEE754_MUL_32(fA3, fB3)

AB12 = IEEE754_ADD_32(A1B1, A2B2)
AB23 = IEEE754_ADD_32(AB12, A3B3)

result_754 = binary_to_float_32(AB23)

print('result_754 = {}'.format(result_754))

#True Result
floatA1 = binary_to_float(fA1)
floatB1 = binary_to_float(fB1)
floatA2 = binary_to_float(fA2)
floatB2 = binary_to_float(fB2)
floatA3 = binary_to_float(fA3) 
floatB3 = binary_to_float(fB3)

print(floatA1)
print(floatB1)
print(floatA2)
print(floatB2)
print(floatA3)
print(floatB3)


TAB1 = floatA1 * floatB1
TAB2 = floatA2 * floatB2
TAB3 = floatA3 * floatB3

True_result = TAB1 + TAB2 + TAB3

if True_result > 3.4028234663852886e+38:
    True_result = 3.4028234663852886e+38
elif True_result < -3.4028234663852886e+38:
    True_result = -3.4028234663852886e+38

print('True_result = {}'.format(True_result))


result_754 = 2825014528.0
-22854.162109375
-49848.8359375
56235.515625
30108.962890625
3032.00830078125
-2451.066162109375
True_result = 2825014777.618271


In [42]:
def OUTPUT(sign_result, exponent_result, mantissa_result):
    #changed
    #需要增加一个计算00停止的条件
    while mantissa_result[0:3] == "000" or mantissa_result[0:3] == "111":
        mantissa_result = str(mantissa_result[1:26]) + '0'  # 数据左移 指数减少
        exponent_result = str(bin((int(exponent_result, 2) - 1) & 0x1FF))[2:].zfill(9) # 指数位减 1
        if(mantissa_result == '1' * 26 or mantissa_result == '0' * 26):
            exponent_result = '100000000'
            break

    while mantissa_result[0:2] == "01" or mantissa_result[0:2] == "10":
        mantissa_result = mantissa_result[0] + mantissa_result[0:25] # 数据右移 指数增加
        exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1

    exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1
    exponent_result = str(bin((int(exponent_result, 2) + 1) & 0x1FF))[2:].zfill(9) # 指数位加 1
        
    exponent_result, mantissa_result = exponent_judge(exponent_result, mantissa_result)

    #指数转换
    if exponent_result[0] == '1':
        exponent_result = '0' + exponent_result[2:9]

    else:
        exponent_result = '1' + exponent_result[2:9]

    #补码转原码
    if sign_result == '1':
        # 对补码的符号位取反
        inverted = "".join("1" if bit == "0" else "0" for bit in mantissa_result)  
        # 加1得到原码
        mantissa_result = bin(int(inverted, 2) + 1)[2:].zfill(26)
    else:
        mantissa_result = mantissa_result

    result = binary_to_float(sign_result + exponent_result + mantissa_result[3: 26])
    return result

result = OUTPUT(Signal, Exponent, Mantissa)

#结果对比

result_754_sign,  result_754_exponent,  result_754_mantissa   =  float_to_binary(result_754)
result_sign,      result_exponent,      result_mantissa       =  float_to_binary(result)

print('对比结果')
print(True_result)
print(result_754)
print(result)

Error_754 = abs(result - result_754)
Error_True = abs(result - True_result)
Error_inner = abs(result_754 - True_result)
precision_Ture = abs(result     - True_result )/abs(True_result + 0.0000000000001) * 100
precision_754  = abs(result     - result_754  )/abs(result_754  + 0.0000000000001) * 100
precision_inner= abs(result_754 - True_result )/abs(True_result + 0.0000000000001) * 100


print('Error_754        = {}'.format(Error_754))
print('precision_754    = {}'.format(precision_754))
print('Error_True       = {}'.format(Error_True))
print('precision_Ture   = {}'.format(precision_Ture))
print('Error_inner      = {}'.format(Error_inner))
print('precision_inner  = {}'.format(precision_inner))

对比结果
2825014777.618271
2825014528.0
2825013760.0
Error_754        = 768.0
precision_754    = 2.718570090128754e-05
Error_True       = 1017.6182708740234
precision_Ture   = 3.602169726460556e-05
Error_inner      = 249.61827087402344
precision_inner  = 8.835998765446211e-06
